In [ ]:
import math
import csv
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

from collections import OrderedDict, deque
from torch.distributions import Categorical
from IPython.display import clear_output, HTML, display
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib import animation

# ---------------------------------------------------------------------
#  Setup
# ---------------------------------------------------------------------

torch.set_default_dtype(torch.float32)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
rng = np.random.default_rng(123)
torch.manual_seed(123)

# ---------------------------------------------------------------------
#  Environment
# ---------------------------------------------------------------------

min_position = -0.99
max_position =  0.99
max_speed    =  0.07
dt           =  0.1
force_mag    =  0.001
gravity      =  0.0025

min_start_pos = 0.67
max_start_pos = 0.77
max_start_vel = 0.01

state_low  = torch.tensor([min_position, -max_speed], device=device)
state_high = torch.tensor([max_position,  max_speed], device=device)
state_mid  = 0.5 * (state_high + state_low)
state_span = 0.5 * (state_high - state_low)

def normalize_state(states: torch.Tensor) -> torch.Tensor:
    """Scale (pos, vel) to [-1, 1] for the RFF layer."""
    return (states - state_mid) / state_span

box_area   = (max_position - min_position) * (2 * max_speed)


def tg_alpha(x: torch.Tensor) -> torch.Tensor:
    return 0.1 * (
        2 * x / (1 - x**2)
        - 2 * math.pi * torch.sin(2 * math.pi * x)
        - 8 * math.pi * torch.sin(4 * math.pi * x)
    )


def total_horizontal_force(a: torch.Tensor, x: torch.Tensor) -> torch.Tensor:
    # a in {0,1}
    return (a * 2 - 1) * force_mag + tg_alpha(x) * (-gravity)


def next_state(s: torch.Tensor, a: torch.Tensor) -> torch.Tensor:
    """
    s: (N,2) with (x,v)
    a: (N,) 0 or 1
    """
    x = s[:, 0]
    v = s[:, 1]
    f = total_horizontal_force(a.float(), x)

    v = torch.clamp(v + f * dt, -max_speed, max_speed)
    x = torch.clamp(x + v * dt, min_position, max_position)

    return torch.stack([x, v], dim=1)


def sample_proposal(n: int) -> torch.Tensor:
    """Uniform proposal over the state box."""
    u = torch.rand(n, 2, device=device)
    s = state_low + u * (state_high - state_low)
    return s


def sample_start(n: int) -> torch.Tensor:
    """Initial distribution d0."""
    sign = torch.where(
        torch.rand(n, device=device) < 0.5,
        -torch.ones(n, device=device),
        torch.ones(n, device=device)
    )
    pos = sign * torch.rand(n, device=device) * (max_start_pos - min_start_pos) + sign * min_start_pos
    vel = (torch.rand(n, device=device) * 2 - 1) * max_start_vel

    return torch.stack([pos, vel], dim=1)


def render_state_distribution_video(frames, extent, norm, interval_ms=200, save_path=None):
    """Render an HTML5 video showing how the density evolves over time."""
    if not frames:
        return

    fig, ax = plt.subplots(figsize=(8, 6))
    ax.set_xlabel('Position')
    ax.set_ylabel('Velocity')
    im = ax.imshow(
        frames[0][1].T,
        extent=extent,
        origin='lower',
        cmap='gray',
        aspect='auto',
        norm=norm
    )
    title = ax.set_title(f'Learned Distribution (Iteration {frames[0][0]})')

    def update(frame):
        iteration, density_values = frame
        im.set_data(density_values.T)
        title.set_text(f'Learned Distribution (Iteration {iteration})')
        return im,

    anim = animation.FuncAnimation(
        fig,
        update,
        frames=frames,
        interval=interval_ms,
        repeat=False
    )
    fps = max(1, int(1000 / interval_ms)) if interval_ms > 0 else 5
    if save_path:
        target_path = str(save_path)
        if not target_path.lower().endswith('.gif'):
            target_path = f"{target_path}.gif"
        writer = animation.PillowWriter(fps=fps)
        anim.save(target_path, writer=writer)
    html = HTML(anim.to_jshtml())
    plt.close(fig)
    display(html)

# ---------------------------------------------------------------------
#  Fixed policy
# ---------------------------------------------------------------------

class PolicyNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self._fc_policy = nn.Sequential(
            nn.Linear(2, 128),
            nn.Tanh(),
            nn.Linear(128, 128),
            nn.Tanh(),
            nn.Linear(128, 2)
        )

    def forward(self, x):
        return self._fc_policy(x)


policy_net = PolicyNetwork().to(device)

# >>> EDIT THIS PATH IF NEEDED <<<
state_dict = torch.load(
    r"/content/net.pth",
    map_location=device
)
policy_state = OrderedDict(
    {k.replace('_fc_policy.', ''): v
     for k, v in state_dict.items()
     if k.startswith('_fc_policy')}
)
policy_net._fc_policy.load_state_dict(policy_state)
policy_net.eval()


@torch.no_grad()
def sample_actions(s: torch.Tensor) -> torch.Tensor:
    logits = policy_net(s)
    dist = Categorical(logits=logits)
    return dist.sample()


@torch.no_grad()
def policy_probs(states: torch.Tensor) -> torch.Tensor:
    logits = policy_net(states)
    return F.softmax(logits, dim=-1)

# ---------------------------------------------------------------------
#  Density network
# ---------------------------------------------------------------------

class ResidualBlock(nn.Module):
    def __init__(self, d: int, use_ln: bool = False, alpha: float = 0.1):
        super().__init__()
        self.alpha = alpha
        self.ln1 = nn.LayerNorm(d) if use_ln else nn.Identity()
        self.ln2 = nn.LayerNorm(d) if use_ln else nn.Identity()
        self.act = nn.SiLU()
        self.fc1 = nn.Linear(d, d)
        self.fc2 = nn.Linear(d, d)
        nn.init.kaiming_normal_(self.fc1.weight, nonlinearity='relu')
        nn.init.zeros_(self.fc1.bias)
        nn.init.kaiming_normal_(self.fc2.weight, nonlinearity='relu')
        nn.init.zeros_(self.fc2.bias)

    def forward(self, x):
        y = self.act(self.ln1(x))
        y = self.fc1(y)
        y = self.act(self.ln2(y))
        y = self.fc2(y)
        return x + self.alpha * y


class SiLUDensityRes(nn.Module):
    """
    Unnormalized density with residual MLP, plus a global logZ normalizer.
    log_prob(s) = f_theta(s) - logZ
    """
    def __init__(self, hidden: int = 256, blocks: int = 5,
                 use_ln: bool = True, box_area: float = 1.0):
        super().__init__()
        self.inp = nn.Linear(2, hidden)
        nn.init.kaiming_normal_(self.inp.weight, nonlinearity='relu')
        nn.init.zeros_(self.inp.bias)

        self.blocks = nn.Sequential(*[
            ResidualBlock(hidden, use_ln=use_ln, alpha=0.1)
            for _ in range(blocks)
        ])

        self.out = nn.Linear(hidden, 1)
        nn.init.kaiming_normal_(self.out.weight, nonlinearity='relu')
        nn.init.zeros_(self.out.bias)

        with torch.no_grad():
            self.out.bias.fill_(-math.log(box_area))

        self.register_buffer("logZ", torch.zeros(1))

    def log_prob(self, x: torch.Tensor) -> torch.Tensor:
        h = self.inp(x)
        h = self.blocks(h)
        return self.out(h).squeeze(-1) - self.logZ

    def forward(self, x):
        return self.log_prob(x)

# ---------------------------------------------------------------------
# ---------------------------------------------------------------------
# Median heuristic for RFF frequencies
with torch.no_grad():
    median_sample = sample_proposal(4096)
    median_sample_norm = normalize_state(median_sample)
    pairwise_distances = torch.pdist(median_sample_norm, p=2)
    median_distance = pairwise_distances.median().item()


heuristic_multiplier = 4.0
rff_sigma_base = 1.0 / max(median_distance, 1e-6)  * heuristic_multiplier
rff_sigma_low = 0.1 * rff_sigma_base
rff_sigma_mid = 1.0 * rff_sigma_base
rff_sigma_high = 40.0 * rff_sigma_base

print(f"Median normalized distance: {median_distance:.6f}")
print(f"RFF sigma base set to {rff_sigma_base:.6f}")
print(f"Low/Mid/High sigmas: {rff_sigma_low:.6f}, {rff_sigma_mid:.6f}, {rff_sigma_high:.6f}")

#  RFF sampler (3 bands with your hyperparameters)
# ---------------------------------------------------------------------

class RFFSampler2D:
    def __init__(self,
                 sigma_base: float = None,
                 device=None,
                 dtype=torch.float32):
        self.device = device or torch.device('cpu')
        self.dtype = dtype

        if sigma_base is None:
            if 'rff_sigma_base' not in globals():
                raise ValueError("Compute rff_sigma_base via the median heuristic cell first.")
            sigma_base = float(rff_sigma_base)

        self._set_sigmas(sigma_base)

        print("Initialized RFF Sampler with equal-probability frequency bands:")
        print(f"  LOW : sigma={self.sigma_low.item():.4f}")
        print(f"  MID : sigma={self.sigma_mid.item():.4f}")
        print(f"  HIGH: sigma={self.sigma_high.item():.4f}")

        self.omegas = None
        self.biases = None

    def _set_sigmas(self, sigma_base: float):
        sigma_tensor = torch.tensor(sigma_base, device=self.device, dtype=self.dtype)
        self.sigma_base = sigma_tensor
        self.sigma_low = 0.1 * sigma_tensor
        self.sigma_mid = sigma_tensor
        self.sigma_high = 10.0 * sigma_tensor

    def update_sigma_base(self, sigma_base: float):
        self._set_sigmas(sigma_base)
        # clear cached banks so fresh frequencies reflect the new scale
        self.omegas = None
        self.biases = None
        print(f"[RFF] Updated sigma base -> {self.sigma_base.item():.4f}")

    @torch.no_grad()
    def _band(self, sigma, K, device, dtype):
        return torch.normal(0.0, sigma, (K, 2), device=device, dtype=dtype)

    @torch.no_grad()
    def sample_batch(self, B: int, device=None, dtype=None):
        """
        Sample B RFFs using an equal-probability mixture of low/mid/high frequencies.
        """
        device = device or self.device
        dtype = dtype or self.dtype

        B1 = B // 3
        B2 = B // 3
        B3 = B - B1 - B2

        O1 = self._band(self.sigma_low,  B1, device, dtype)
        O2 = self._band(self.sigma_mid,  B2, device, dtype)
        O3 = self._band(self.sigma_high, B3, device, dtype)

        self.omegas = torch.cat([O1, O2, O3], dim=0)            # (B,2)
        self.biases = 2 * torch.pi * torch.rand(B, device=device, dtype=dtype)
        return self.omegas, self.biases


    @torch.no_grad()
    def features(self, states: torch.Tensor,
                 omegas: torch.Tensor = None,
                 biases: torch.Tensor = None) -> torch.Tensor:
        if omegas is None or biases is None:
            if self.omegas is None or self.biases is None:
                raise ValueError("Call sample_batch(B) first or supply omegas/biases.")
            omegas, biases = self.omegas, self.biases

        states_norm = normalize_state(states)
        proj = states_norm @ omegas.T
        return torch.cos(proj + biases)

# ---------------------------------------------------------------------
#  Sigma refresh helper
# ---------------------------------------------------------------------

@torch.no_grad()
def estimate_sigma_from_density(density: nn.Module,
                                sample_size: int = 4096,
                                pool_multiplier: int = 4) -> tuple[float, float]:
    total = sample_size * max(1, pool_multiplier)
    pool_states = sample_proposal(total)
    logw = density.log_prob(pool_states)
    logw = logw - logw.max()
    weights = torch.exp(logw)
    probs = weights / weights.sum()
    idx = torch.multinomial(probs, num_samples=sample_size, replacement=True)
    selected = pool_states[idx]
    normalized = normalize_state(selected)
    distances = torch.pdist(normalized, p=2)
    median_distance = distances.median().item()
    sigma = 1.0 / max(median_distance, 1e-6)
    return sigma, median_distance

# ---------------------------------------------------------------------
#  U-gradient estimator
# ---------------------------------------------------------------------

def rff_expect_next_feature(states: torch.Tensor,
                            probs: torch.Tensor,
                            sampler: RFFSampler2D,
                            omega: torch.Tensor,
                            b: torch.Tensor) -> torch.Tensor:
    """
    E_a[ f(s⁺) ] under policy.
    """
    N = states.size(0)
    a0 = torch.zeros(N, dtype=torch.long, device=states.device)
    a1 = torch.ones(N,  dtype=torch.long, device=states.device)

    sp0 = next_state(states, a0)
    sp1 = next_state(states, a1)

    F_sp0 = sampler.features(sp0, omega, b)
    F_sp1 = sampler.features(sp1, omega, b)

    return probs[:, 0:1] * F_sp0 + probs[:, 1:2] * F_sp1


def estimate_R_and_grad(N: int,
                        gamma: float,
                        sampler: RFFSampler2D,
                        omega: torch.Tensor,
                        b: torch.Tensor,
                        density: nn.Module,
                        box_area: float):
    """
    Compute R_f and its gradient factor via U-gradient.
    """
    # ----- Batch S for R_f -----
    S  = sample_proposal(N)
    S0 = sample_start(N)
    pS = policy_probs(S)

    F_S   = sampler.features(S,  omega, b)        # (N,B)
    EF_Sp = rff_expect_next_feature(S, pS, sampler, omega, b)  # (N,B)
    F_S0  = sampler.features(S0, omega, b)        # (N,B)

    with torch.no_grad():
        logw_S = density.log_prob(S) + math.log(box_area)
        w_S = torch.exp(logw_S)

        R_all = (w_S.unsqueeze(1) * (F_S - gamma * EF_Sp)).mean(dim=0) \
                - (1.0 - gamma) * F_S0.mean(dim=0)
        R_all = R_all.detach()

    # ----- Independent St for grad -----
    St  = sample_proposal(N)
    pSt = policy_probs(St)

    F_St   = sampler.features(St,  omega, b)
    EF_Stp = rff_expect_next_feature(St, pSt, sampler, omega, b)

    with torch.no_grad():
        A_f = (F_St - gamma * EF_Stp).detach()

    logwt = density.log_prob(St) + math.log(box_area)
    wt = torch.exp(logwt)

    grad_all = (A_f * wt.unsqueeze(1)).mean(dim=0)

    return R_all, grad_all

# ---------------------------------------------------------------------
#  Train step (fixed RFF bank)
# ---------------------------------------------------------------------

def train_step(N: int,
               gamma: float,
               sampler: RFFSampler2D,
               density: nn.Module,
               optimizer: torch.optim.Optimizer,
               box_area: float,
               B: int):

    omega, b = sampler.sample_batch(B, device=device, dtype=torch.float32)

    R_all, grad_all = estimate_R_and_grad(
        N=N, gamma=gamma, sampler=sampler,
        omega=omega, b=b,
        density=density, box_area=box_area
    )

    loss = 2.0 * (R_all * grad_all).mean()

    optimizer.zero_grad()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(density.parameters(), max_norm=5.0)
    optimizer.step()

    return loss.item()

# ---------------------------------------------------------------------
#  Training loop
# ---------------------------------------------------------------------

def train(T=10000, N=15000,
          B=1024,
          gamma=0.99, lr=1e-3,
          print_every=50, visualize_every=200,
          logZ_tau=0.1,
          make_video=True,
          max_video_frames=200,
          video_path=None,
          sigma_update_every=500,
          sigma_sample_size=4096):

    # visualization grid
    pos_grid = torch.linspace(min_position, max_position, 400, device=device)
    vel_grid = torch.linspace(-max_speed,  max_speed,  400, device=device)
    P, V = torch.meshgrid(pos_grid, vel_grid, indexing='ij')
    grid_states = torch.stack((P.flatten(), V.flatten()), dim=1)
    plot_extent = [min_position, max_position, -max_speed, max_speed]

    # CSV logging
    csv_filename = 'training_metrics.csv'
    csv_file = open(csv_filename, 'w', newline='')
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['Iteration', 'Loss', 'MeanWeight', 'BowlWeight', 'BridgeWeight'])

    # modules
    density = SiLUDensityRes(hidden=256, blocks=5, use_ln=True, box_area=box_area).to(device)
    optimizer = torch.optim.Adam(density.parameters(), lr=lr)
    sampler = RFFSampler2D(device=device, sigma_base=float(rff_sigma_base))
    plot_norm = colors.PowerNorm(gamma=0.4, vmin=0.0, vmax=100.0)
    video_frames = deque(maxlen=max_video_frames if max_video_frames else None) if make_video else None

    hist = []
    for t in range(1, T + 1):
        L_val = train_step(
            N=N, gamma=gamma,
            sampler=sampler, density=density,
            optimizer=optimizer, box_area=box_area,
            B=B
        )
        hist.append(L_val)

        if sigma_update_every and t % sigma_update_every == 0:
            new_sigma, median_dist = estimate_sigma_from_density(
                density,
                sample_size=sigma_sample_size
            )
            sampler.update_sigma_base(new_sigma)
            globals()['rff_sigma_base'] = new_sigma
            globals()['rff_sigma_low'] = 0.1 * new_sigma
            globals()['rff_sigma_mid'] = new_sigma
            globals()['rff_sigma_high'] = 10.0 * new_sigma
            print(
                f"[sigma] Iter {t}: base={new_sigma:.6f} (median_dist={median_dist:.6f})"
            )

        # visualization
        if t % visualize_every == 0:
            clear_output(wait=True)
            with torch.no_grad():
                logd = density.log_prob(grid_states)
                d_values = torch.exp(logd).cpu().numpy().reshape(400, 400)
            if video_frames is not None:
                video_frames.append((t, d_values.copy()))
            print(f"Iteration {t}/{T}: L_est={L_val:.6f}")
            fig, ax = plt.subplots(figsize=(8, 6))
            im = ax.imshow(
                d_values.T,
                extent=plot_extent,
                origin='lower',
                cmap='gray',
                aspect='auto',
                norm=plot_norm
            )
            plt.colorbar(im, ax=ax)
            ax.set_xlabel('Position')
            ax.set_ylabel('Velocity')
            ax.set_title(f'Learned Distribution (Iteration {t})')
            plt.show()
            plt.close()

        # logging + logZ update
        if t % print_every == 0:
            with torch.no_grad():
                s_chk  = sample_proposal(200_000)
                logw_c = density.log_prob(s_chk) + math.log(box_area)

                logw_clamped = torch.clamp(logw_c, min=-8.0, max=8.0)
                w_c = torch.exp(logw_clamped)

                # numerically stable estimate of E_m[w]

                shift = logw_c.max()
                w_mean = (torch.exp(logw_c - shift).mean() * torch.exp(shift)).item()

                # logZ := logZ + τ * log(E_m[w]) so mass is nudged toward 1
                delta_logZ = math.log(max(w_mean, 1e-6))
                density.logZ.add_(logZ_tau * delta_logZ)


            print(f"Iteration {t}/{T}: L_est={L_val:.6f} | E_m[w]≈{w_mean:.3f}", end='', flush=True)

            with torch.no_grad():
                bowl_mask = s_chk[:, 0].abs() > 0.6
                bridge_mask = s_chk[:, 0].abs() < 0.3
                w_bowls = w_c[bowl_mask].mean().item() if bowl_mask.sum() > 0 else 0.0
                w_bridge = w_c[bridge_mask].mean().item() if bridge_mask.sum() > 0 else 0.0
                print(f"\n--- Weight Check (Iter {t}): "
                      f"Avg Bowl Weight = {w_bowls:.4f} | Avg Bridge Weight = {w_bridge:.4f} ---")
                csv_writer.writerow([t, L_val, w_mean, w_bowls, w_bridge])
                csv_file.flush()

    csv_file.close()
    print(f"\nTraining complete. Metrics saved to {csv_filename}")
    if video_frames:
        render_state_distribution_video(
            list(video_frames),
            extent=plot_extent,
            norm=plot_norm,
            save_path=video_path
        )
    return hist, density

is_discrete: True
----------------------------------------
| Iteration 0
| Actor Loss: -0.698, Critic Loss: 0.291, Alpha: 0.9997
----------------------------------------
----------------------------------------
| Iteration 1000
| Actor Loss: -3.265, Critic Loss: 0.040, Alpha: 0.8976
----------------------------------------
----------------------------------------
| Iteration 2000
| Actor Loss: -5.881, Critic Loss: 0.037, Alpha: 0.7990
----------------------------------------
----------------------------------------
| Iteration 3000
| Actor Loss: -8.242, Critic Loss: 0.028, Alpha: 0.5756
----------------------------------------
----------------------------------------
| Iteration 4000
| Actor Loss: -9.757, Critic Loss: 0.034, Alpha: 0.3270
----------------------------------------
----------------------------------------
| Iteration 5000
| Actor Loss: -10.680, Critic Loss: 0.045, Alpha: 0.1682
----------------------------------------
--- Evaluation | Iteration 5000 | Avg Reward: 26.60 ± 

In [ ]:
# save as make_videos.py and run: python make_videos.py
import glob
import imageio.v2 as imageio

def build_video(pattern, output, fps=5):
    frames = sorted(glob.glob(pattern))
    if not frames:
        print(f"No frames matching {pattern}")
        return
    with imageio.get_writer(output, fps=fps) as writer:
        for f in frames:
            writer.append_data(imageio.imread(f))
    print(f"Wrote {output} ({len(frames)} frames)")

build_video("plots/state_distribution/state_dist_*.png", "state_distribution.mp4")
build_video("plots/policy/policy_*.png", "policy.mp4")
build_video("plots/value/value_*.png", "value.mp4")

Wrote state_distribution.mp4 (200 frames)


Wrote policy.mp4 (200 frames)
Wrote value.mp4 (200 frames)
